In [2]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import h5py
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
import pywt
from plotly import tools
from plotly.offline import init_notebook_mode, iplot, plot
# import cufflinks as cf
import plotly.graph_objs as go
from datetime import datetime
from datetime import timedelta
import warnings
import time
warnings.filterwarnings('ignore')

In [3]:
X_train = h5py.File("original_data/X_train.h5", "r")
y_train_ = pd.read_csv("original_data/y_train.csv").as_matrix()[:, 1].squeeze()
df_train = pd.DataFrame(data=X_train["features"][:])

In [4]:
def label_columns(df):
    l_columns = ['num_pso', 
             'mean_amp_pso',
             'mean_dur_pso',
              'amp_cso',
              'dur_cso',
              'curr_sleep_stage',
              'time_since_sleep',
              'time_in_ds',
              'time_in_ls',
              'time_in_rs',
              'time_in_ws']
    for i in range(12, 1261+1):
        l_columns.append('eeg_signal_%s'%(i-12+1))
    df.columns = l_columns

    return df
df_train=label_columns(df_train)

In [5]:
eegcolumns = ['eeg_signal_%s'%(i-12+1) for i in range(12, 1261+1)]
cols=list(df_train.columns)
selected_var=[c for c in eegcolumns]

X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(df_train[selected_var], y_train_, test_size=0.10,random_state=0, stratify=y_train_)

In [6]:
n_image_train=128
scales = range(1,1251)
start = time.time()
Array_images_train=np.ndarray(shape=(n_image_train, 1250, 1250,1))
for i in range(0,n_image_train):
    Array_images_train[i, :, :,0] = (abs(pywt.cwt(X_train_cnn.iloc[i,:], scales,"morl", 1/125)[0]))**2
end = time.time()
print(end - start)

234.02365946769714


In [7]:
n_image_test=128
scales = range(1,1251)
start = time.time()
Array_images_test=np.ndarray(shape=(n_image_test, 1250, 1250,1))
for i in range(0,n_image_test):
    Array_images_test[i, :, :,0] = (abs(pywt.cwt(X_val_cnn.iloc[i,:], scales,"morl", 1/125)[0]))**2
end = time.time()
print(end - start)

236.86612725257874


In [8]:
x_train = Array_images_train
y_train_cnn_cut = y_train_cnn[0:n_image_train]
x_test = Array_images_test
y_test_cnn_cut = y_val_cnn[0:n_image_test]

In [9]:
import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import History 
history = History()
 
img_x = 1250
img_y = 1250
img_z = 1
input_shape = (img_x, img_y, img_z)
 
num_classes = 3
batch_size = 16
epochs = 5
 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
    
y_train = keras.utils.to_categorical(y_train_cnn_cut, num_classes)
y_test = keras.utils.to_categorical(y_test_cnn_cut, num_classes)
 
 
model = Sequential()

model.add(Conv2D(4, kernel_size=(5, 5),activation='relu', input_shape=input_shape))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(num_classes, activation='softmax'))
 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])
 
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))

Using TensorFlow backend.


Train on 128 samples, validate on 128 samples
Epoch 1/5
128/128 [==============================] - 41s 319ms/step - loss: 2865766.8638 - accuracy: 0.4062 - val_loss: 2862329.5469 - val_accuracy: 0.3516
Epoch 2/5
128/128 [==============================] - 38s 301ms/step - loss: 1866672.7031 - accuracy: 0.4453 - val_loss: 1247174.3359 - val_accuracy: 0.4297
Epoch 3/5
128/128 [==============================] - 38s 297ms/step - loss: 423290.3867 - accuracy: 0.4766 - val_loss: 1438629.2891 - val_accuracy: 0.2969
Epoch 4/5
128/128 [==============================] - 38s 297ms/step - loss: 225575.8633 - accuracy: 0.5547 - val_loss: 228301.4648 - val_accuracy: 0.3516
Epoch 5/5
128/128 [==============================] - 38s 296ms/step - loss: 51973.5015 - accuracy: 0.6328 - val_loss: 76440.0083 - val_accuracy: 0.4375
Train loss: 26853.267333984375, Train accuracy: 0.6796875
Test loss: 76440.0087890625, Test accuracy: 0.4375
